In [33]:

import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

In [103]:
x = pd.read_csv("Cary_Weather_Data.csv")
x['DATE'] = pd.to_datetime(x[['YEAR', 'MONTH', 'DAY']])
x = x.drop(columns=['YEAR','MONTH','DAY'])
#x = x.drop(columns=['Humidity','Precipitation','Wind Speed'])
x = x[['DATE', 'Temperature', 'Humidity', 'Precipitation', 'Wind Speed']]
x.head()
x.set_index('DATE', inplace=True)
x.head()


,Temperature,Humidity,Precipitation,Wind Speed
DATE,,,,
1981-07-01,15.12,10.68,6.65,3.59
1981-07-02,7.53,13.12,10.68,1.64
1981-07-03,7.32,13.79,37.42,2.43
1981-07-04,10.43,14.40,8.41,2.29
1981-07-05,13.13,15.20,4.13,1.58


In [80]:
scaler = StandardScaler()
x_scale = scaler.fit_transform(x)
x_scale = pd.DataFrame(x_scale, columns=x.columns, index=x.index)
x_scale.head()

,Temperature,Humidity,Precipitation,Wind Speed
DATE,,,,
1981-07-01,0.926990,0.385333,0.478536,1.918945
1981-07-02,-1.305674,0.925994,1.044274,-0.352127
1981-07-03,-1.367448,1.074455,4.798080,0.567948
1981-07-04,-0.452614,1.209620,0.725608,0.404897
1981-07-05,0.341614,1.386887,0.124774,-0.422006


In [36]:
def create_dataset(X, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(X.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [73]:
time_steps = 1

X, y = create_dataset(x_scale, time_steps)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123, shuffle=False)
X_train.shape
X_val.shape

(3106, 1, 4)

In [38]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=[time_steps, 4]))
model.add(tf.keras.layers.LSTM(64))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, 'linear'))

cp = tf.keras.callbacks.ModelCheckpoint('model', save_best_only=True)
#model.compile(loss='mse', optimizer='adam', metrics=['rmse'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                17664     
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 18193 (71.07 KB)
Trainable params: 18193 (71.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
model.compile(loss='Huber', optimizer='adam', metrics=['mae'])

In [40]:
cp = tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True)
history = model.fit(X_train, y_train, epochs=10, callbacks=cp)

Epoch 1/10
389/389 [==============================] - 2s 3ms/step - loss: 0.3620 - mae: 0.7119
Epoch 2/10
389/389 [==============================] - 1s 3ms/step - loss: 0.3600 - mae: 0.7090
Epoch 3/10
389/389 [==============================] - 2s 4ms/step - loss: 0.3598 - mae: 0.7086
Epoch 4/10
389/389 [==============================] - 2s 4ms/step - loss: 0.3597 - mae: 0.7088
Epoch 5/10
389/389 [==============================] - 1s 4ms/step - loss: 0.3597 - mae: 0.7089
Epoch 6/10
389/389 [==============================] - 2s 5ms/step - loss: 0.3596 - mae: 0.7089
Epoch 7/10
389/389 [==============================] - 2s 5ms/step - loss: 0.3596 - mae: 0.7087
Epoch 8/10
389/389 [==============================] - 2s 4ms/step - loss: 0.3596 - mae: 0.7090
Epoch 9/10
389/389 [==============================] - 1s 3ms/step - loss: 0.3594 - mae: 0.7087
Epoch 10/10
389/389 [==============================] - 1s 3ms/step - loss: 0.3595 - mae: 0.7087


In [41]:
matplotlib.interactive(True)
%matplotlib qt
plt.plot(history.history['loss'], label='Training Loss')
#plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [111]:
prediction = model.predict(X_val)
prediction_reshape = prediction[:, 0]
x_val_reshz = X_val[:, 0, 0]

98/98 [==============================] - 0s 691us/step


In [108]:
time_val = x.index[-len(X_val):]

In [113]:
plt.plot(x_scale['Temperature'], color='green')
plt.plot(time_val, x_val_reshz, label='Actual', color='blue')
plt.plot(time_val, prediction_reshape, label='Prediction', color='orange')
plt.show()